# Notion API

In [2]:
import requests

# Notion APIトークン
token = 'secret_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# データベースID
database_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [3]:
# ページの取得
def get_pages(token, database_id):
    # ヘッダーにAuthorizationを設定
    headers = {
        'Authorization': f'Bearer {token}',
        'Notion-Version': '2022-06-28'  # Notion APIバージョン
    }

    # フィルタリング条件を設定
    filter_condition = {
        "property": "Published",  # チェックボックスのプロパティ名
        "checkbox": {
            "equals": True  # Trueであるアイテムをフィルタリング
        }
    }

    url = f'https://api.notion.com/v1/databases/{database_id}/query'
    data = {
        "filter": filter_condition
    }
    response = requests.post(url, headers=headers, json=data)
    pages = response.json()
    return pages


In [4]:
# ページの内容を取得
def get_page_data(token, page_id):
    # Notion APIエンドポイント
    endpoint = f"https://api.notion.com/v1/blocks/{page_id}/children"

    # リクエストヘッダー
    headers = {
        "Authorization": f"Bearer {token}",
        "Notion-Version": "2022-06-28",
        "Content-Type": "application/json",
    }

    # APIリクエストを送信してページの内容を取得
    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        page_data = response.json()
        return page_data
    else:
        print(f"エラー：{response.status_code}")
        return None

In [5]:
# ページの内容からテキストと子ページのタイトルを抽出
def extract_text_and_titles(data):
    text = ""

    for block in data['results']:
        if block['type'] == 'paragraph':
            text += block['paragraph']['rich_text'][0]['text']['content'] + '\n'
        elif block['type'] == 'heading_1':
            text += block['heading_1']['rich_text'][0]['text']['content'] + '\n'
        elif block['type'] == 'heading_2':
            text += block['heading_2']['rich_text'][0]['text']['content'] + '\n'
        elif block['type'] == 'heading_3':
            text += block['heading_3']['rich_text'][0]['text']['content'] + '\n'
        elif block['type'] == 'bulleted_list_item':
            text += block['bulleted_list_item']['rich_text'][0]['text']['content'] + '\n'
        elif block['type'] == 'child_page':
            text += block['child_page']['title'] + '\n'

    # textの最後の改行を削除
    text = text.rstrip('\n')
    return text

In [6]:
def get_notion_info(token, database_id):
    # 保存用の配列
    contents = []  
    # データベースからページを全て取得
    pages = get_pages(token, database_id)   
    # ページデータを表示
    for page in pages['results']:
        page_url = page['url']
        page_id = page['id']
        page_data = get_page_data(token, page_id)
        content = extract_text_and_titles(page_data)
        # URLとページの内容を配列に追加
        contents.append([page_url, content])

    return contents

In [7]:
contents = get_notion_info(token, database_id)

In [8]:
contents

[['https://www.notion.so/Notion-5ac93f250a764049880e2d1e4bcebebe',
  'Notionで新規ページを作成する権限を付与してもらう方法は以下の手順に従います。\nワークスペースの管理者に連絡\n最初に、ワークスペースの管理者（北爪さん）に連絡し、ページ作成の権限を要求します。\n権限レベルの確認\n管理者は、あなたのアカウントがどの権限レベルにあるかを確認します。Notionでは、一般的に「メンバー」または「管理者」になる必要があります。\n権限の変更\n管理者は、ワークスペースの設定にアクセスし、あなたのアカウントの権限を「メンバー」や「管理者」に変更できます。\nページ作成の確認\n権限が変更された後、新しいページを作成して、権限が正しく設定されていることを確認します。\n追加の設定\n必要に応じて、特定のページやフォルダーに対する追加のアクセス権を設定することができます。'],
 ['https://www.notion.so/544110ecdefc441282d1731132e940fb',
  '経費精算の手順は以下の通りです。\n経費の規定確認\n精算可能な経費とその条件を、会社の経費規定やマニュアルで確認します。\nレシートや領収書の収集\n精算する経費に関するレシートや領収書を収集します。\n経費報告書の作成\n所定の経費報告書に、日付、経費の種類、金額、経費の目的や理由を記入します。デジタルのシステムを使用する場合は、オンラインで入力します。\n添付資料の準備\nレシートや領収書のコピー、必要に応じて移動経路や時間帯の説明を添付します。\n上司への提出・承認依頼\n報告書と添付資料を上司に提出し、承認を依頼します。\n経理部門への提出\n上司の承認を得た後、報告書と添付資料を経理部門に提出します。\n精算の処理待ち\n経理部門での精算処理を待ちます。この段階で不明点があれば経理部門から問い合わせが来ることがあります。\n精算金の受領\n精算が完了し、指定された方法（銀行振込、給与との併給など）で精算金を受領します。\n注意点\nレシートや領収書は紛失しないように注意し、必要な情報が全て記載されていることを確認します。\n精算申請はできるだけ早めに行うことが望ましいです。'],
 ['

# Cognitive Search

In [9]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter

# 文章とメタデータを受け取り、Documentのリストを返す
def txtToDocs(content:str, metadata:dict):
    docs = [Document(page_content=content, metadata=metadata)] # Documentのリストを作成
    chunk_size=1000 # Documentの文字数の設定
    separator='' # Documentの区切り文字の設定(''の場合、chunk_sizeで指定した文字数で区切る)
    chunk_overlap=50 # 分割した文章のオーバーラップする文字数の設定

    # 文章を分割するためのインスタンスを作成
    text_splitter = CharacterTextSplitter(
            chunk_size=chunk_size, 
            separator=separator, 
            chunk_overlap=chunk_overlap
            )
    # Documentのリストを設定したパラメータで分割
    splitted_docs = text_splitter.split_documents(docs)
    return splitted_docs

In [13]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

# ベクターストアのロード
def azureLoad(fields=None):
    embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
        deployment=os.environ['OPENAI_API_EMBEDDING_DEPLOYMENT_NAME'], # モデルのデプロイメント名
        model_name=os.environ['OPENAI_API_EMBEDDING_MODEL_NAME'] # モデル名
    ) # OpenAIのモデルを使用してベクトル化

    # ベクターストアのインスタンスを作成
    vectore_stores: AzureSearch = AzureSearch(
        azure_search_endpoint=os.environ['AZURE_VECTORE_STORES_ADDRESES'],  
        azure_search_key=os.environ['AZURE_VECTORE_STORES_PASSWORD'],  
        index_name=os.environ['AZURE_VECTORE_STORES_INDEX_NAME'],
        embedding_function=embeddings.embed_query,
        fields=fields
    )
    return vectore_stores

In [14]:
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField
)

# ベクターストアへのドキュメントの追加
def azureAddDocuments(documents:list[Document]):
    embeddings: OpenAIEmbeddings = OpenAIEmbeddings() # OpenAIのモデルを使用してベクトル化
    
    # ベクターストアの検索フィールドの設定
    # metadataでフィルタリングする場合は、filterable=Trueを設定する
    fields = [
        SimpleField(
            name="id",
            type=SearchFieldDataType.String,
            key=True,
            filterable=True,
        ),
        SearchableField(
            name="content",
            type=SearchFieldDataType.String,
            searchable=True,
        ),
        SearchField(
            name="content_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=len(embeddings.embed_query("Text")),
            vector_search_configuration="default",
        ),
        SearchableField(
            name="metadata",
            type=SearchFieldDataType.String,
            searchable=True,
        ),
        # フィルタリングしたいmetadataはここに追加する
        SimpleField(
            name="notion_id", # フィルタリングしたいmetadataのキー
            type=SearchFieldDataType.String, # フィルタリングしたいmetadataの型
            filterable=True, # フィルタリング可能にする
        ),
    ]

    vectore_stores = azureLoad(fields=fields) # ベクターストアのロード

    res = vectore_stores.add_documents(documents) # ベクターストアへのドキュメントの追加
    return res

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .envファイルから環境変数を読み込む

for content in contents:
    metadata = {
        "notion_id": content[0],
    }
    docs = txtToDocs(content[1], metadata)
    azureAddDocuments(docs)